In [1]:
import re
import requests
import pandas as pd
from pandas import DataFrame
import numpy as np

In [3]:
for i in [1, 2, 4, 5]:
    m = requests.request("GET", "http://www.mufap.com.pk/nav_returns_performance.php?tab=0" + str (i)).text
    m = m.replace('\n', '').replace('\t', '').replace('\s+', '')
    m = m + "</style>"
    
    x = re.findall('<table id="dataTable" class="tab-headings1 tb-0'+str(i)+'">(.*?)</style>', m)
    r = []
    for y in x:
        tableHeading = re.findall('<td colspan="16">(.*?)</td>', y)[0]
        z = re.findall('<tr class="tab-data1.*?>(.*?)</tr>', y)
        for _ in z:
            q = re.findall('<td.*?>(.*?)</td>', _)
            if i == 2:
#                 if tableHeading[0] is not ' ': tableHeading = ' ' + tableHeading
                q[0] = q[0] + ' ' + tableHeading
#                 q[0] = q[0].replace("-*\)\(", ' ')
#                 q[0] = ' '.join(q[0].split())
#                 print(q)
            else:
                q[0] = q[0].replace("\W", '')
            r.append(q)

    df = DataFrame.from_records(r)
    print(df.shape)
    if len(df.loc[0]) == 13:
        df.columns = ['Fund Name', 'Rating', 'Validity Date', 'NAV', 'YTD', 'MTD', '1 Day', '15 Day', '30 Day', '90 Day', '180 Day', '270 Day', '365 Days']
    elif len(df.loc[0]) == 12:
        df.columns = ['Fund Name', 'Validity Date', 'NAV', 'YTD', 'MTD', '1 Day', '15 Day', '30 Day', '90 Day', '180 Day', '270 Day', '365 Days']
    elif len(df.loc[0]) == 15:
        df.columns = ['Fund Name', 'Validity Date', 'NAV', 'YTD', 'MTD', '1 Day', '15 Day', '30 Day', '90 Day', '180 Day', '270 Day', '365 Days', 'TER', 'MF', 'S&M']
    
    df['Fund Name'] = df['Fund Name'].str.replace('\xa0',' ').str.replace(':', '').str.replace('-', ' ').str.replace('\(', '').str.replace('\)', '').str.replace('\*', '').str.replace('\s\s*', ' ').str.replace('â', ' ').str.replace('', '')
    df['365 Days'] = df['365 Days'].str.replace('\(', '-').str.replace('\)', '')
    df.to_excel('mufap'+str(i)+'.xls')
    
    

(267, 13)
(66, 12)
(9, 12)
(6, 12)
